In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.listdir('/content/drive/MyDrive')

['Colab Notebooks', 'unet_project', 'UNET_BACKUP_ONLY']

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

os.listdir('/content/drive/MyDrive')

['Colab Notebooks', 'unet_project', 'UNET_BACKUP_ONLY', 'dataset_ruin3']

In [3]:
BASE_PATH = "/content/drive/MyDrive"
DATASET_PATH = BASE_PATH + "/dataset_ruin3"

IMAGE_DIR = DATASET_PATH + "/original_images_archive (1)"
MASK_DIR = DATASET_PATH + "/merged_multi_class_masks_archive"

print("Images:", len(os.listdir(IMAGE_DIR)))
print("Masks:", len(os.listdir(MASK_DIR)))

Images: 87
Masks: 87


In [4]:
YOLO_BASE = BASE_PATH + "/Colab Notebooks/Yolo_Detection"

folders = [
    "dataset/images/train",
    "dataset/images/val",
    "dataset/labels/train",
    "dataset/labels/val",
    "runs"
]

import os
for f in folders:
    os.makedirs(os.path.join(YOLO_BASE, f), exist_ok=True)

print("YOLO project folders created")

YOLO project folders created


In [6]:
print("Train images:", len(os.listdir(YOLO_BASE + "/dataset/images/train")))
print("Train labels:", len(os.listdir(YOLO_BASE + "/dataset/labels/train")))
print("Val images:", len(os.listdir(YOLO_BASE + "/dataset/images/val")))
print("Val labels:", len(os.listdir(YOLO_BASE + "/dataset/labels/val")))

Train images: 0
Train labels: 0
Val images: 0
Val labels: 0


In [7]:
print("Sample images:", os.listdir(IMAGE_DIR)[:5])
print("Sample masks:", os.listdir(MASK_DIR)[:5])

Sample images: ['ruin9_3_9.png', 'ruin8_7_2.png', 'ruin6_2_15.png', 'ruin29_6_3.png', 'ruin6_3_9.png']
Sample masks: ['ruin4_1_13_merged_mask.png', 'ruin4_2_14_merged_mask.png', 'ruin6_2_13_merged_mask.png', 'ruin7_6_5_merged_mask.png', 'ruin6_6_1_merged_mask.png']


In [8]:
import cv2
import random
import shutil
from tqdm import tqdm
import os

image_files = sorted(os.listdir(IMAGE_DIR))
random.shuffle(image_files)

split = int(0.8 * len(image_files))
train_files = image_files[:split]
val_files = image_files[split:]

def convert(files, img_out, lbl_out):
    for img_name in tqdm(files):
        img_path = os.path.join(IMAGE_DIR, img_name)

        base = img_name.replace(".png", "")
        mask_name = base + "_merged_mask.png"
        mask_path = os.path.join(MASK_DIR, mask_name)

        if not os.path.exists(mask_path):
            continue

        img = cv2.imread(img_path)
        mask = cv2.imread(mask_path, 0)

        h, w = mask.shape
        _, thresh = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(
            thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
        )

        label_path = os.path.join(lbl_out, base + ".txt")

        with open(label_path, "w") as f:
            for cnt in contours:
                x, y, bw, bh = cv2.boundingRect(cnt)

                if bw < 10 or bh < 10:
                    continue

                xc = (x + bw / 2) / w
                yc = (y + bh / 2) / h
                bw /= w
                bh /= h

                f.write(f"0 {xc} {yc} {bw} {bh}\n")

        shutil.copy(img_path, os.path.join(img_out, img_name))

convert(
    train_files,
    YOLO_BASE + "/dataset/images/train",
    YOLO_BASE + "/dataset/labels/train"
)

convert(
    val_files,
    YOLO_BASE + "/dataset/images/val",
    YOLO_BASE + "/dataset/labels/val"
)

print("Mask → YOLO label conversion DONE (fixed naming)")

100%|██████████| 18/18 [00:11<00:00,  1.51it/s]

Mask → YOLO label conversion DONE (fixed naming)


In [9]:
print("Train images:", len(os.listdir(YOLO_BASE + "/dataset/images/train")))
print("Train labels:", len(os.listdir(YOLO_BASE + "/dataset/labels/train")))
print("Val images:", len(os.listdir(YOLO_BASE + "/dataset/images/val")))
print("Val labels:", len(os.listdir(YOLO_BASE + "/dataset/labels/val")))

Train images: 69
Train labels: 69
Val images: 18
Val labels: 18


In [10]:
!pip install -q ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.3 MB/s eta 0:00:00


In [11]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [12]:
import yaml
import os

DATA_YAML_PATH = YOLO_BASE + "/dataset/data.yaml"

data_yaml = {
    "path": YOLO_BASE + "/dataset",
    "train": "images/train",
    "val": "images/val",
    "nc": 1,
    "names": ["ruin"]
}

with open(DATA_YAML_PATH, "w") as f:
    yaml.dump(data_yaml, f)

print("data.yaml created at:", DATA_YAML_PATH)

data.yaml created at: /content/drive/MyDrive/Colab Notebooks/Yolo_Detection/dataset/data.yaml


In [13]:
import yaml
import os

DATA_YAML_PATH = YOLO_BASE + "/dataset/data.yaml"

data_yaml = {
    "path": YOLO_BASE + "/dataset",
    "train": "images/train",
    "val": "images/val",
    "nc": 1,
    "names": ["ruin"]
}

with open(DATA_YAML_PATH, "w") as f:
    yaml.dump(data_yaml, f)

print("data.yaml created at:", DATA_YAML_PATH)

data.yaml created at: /content/drive/MyDrive/Colab Notebooks/Yolo_Detection/dataset/data.yaml


In [14]:
model = YOLO("yolov8n.pt")

model.train(
    data=DATA_YAML_PATH,
    epochs=50,
    imgsz=640,
    batch=16,
    project=YOLO_BASE + "/yolo_runs",
    name="ruin_detection",
    exist_ok=True
)

Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Colab Notebooks/Yolo_Detection/dataset/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=ruin_detection, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f71fb5dc050>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [15]:
from ultralytics import YOLO

model = YOLO(
    YOLO_BASE + "/yolo_runs/ruin_detection/weights/best.pt"
)

model.predict(
    source=YOLO_BASE + "/dataset/images/val",
    save=True,
    project=YOLO_BASE + "/predictions",
    name="ruin_results",
    exist_ok=True,
    conf=0.25
)


image 1/18 /content/drive/MyDrive/Colab Notebooks/Yolo_Detection/dataset/images/val/ruin29_6_12.png: 640x640 (no detections), 350.3ms
image 2/18 /content/drive/MyDrive/Colab Notebooks/Yolo_Detection/dataset/images/val/ruin29_7_2.png: 640x640 (no detections), 424.3ms
image 3/18 /content/drive/MyDrive/Colab Notebooks/Yolo_Detection/dataset/images/val/ruin29_7_3.png: 640x640 (no detections), 897.9ms
image 4/18 /content/drive/MyDrive/Colab Notebooks/Yolo_Detection/dataset/images/val/ruin3_7_4.png: 640x640 1 ruin, 287.2ms
image 5/18 /content/drive/MyDrive/Colab Notebooks/Yolo_Detection/dataset/images/val/ruin4_1_13.png: 640x640 4 ruins, 293.1ms
image 6/18 /content/drive/MyDrive/Colab Notebooks/Yolo_Detection/dataset/images/val/ruin4_2_14.png: 640x640 2 ruins, 288.9ms
image 7/18 /content/drive/MyDrive/Colab Notebooks/Yolo_Detection/dataset/images/val/ruin4_4_0.png: 640x640 9 ruins, 294.7ms
image 8/18 /content/drive/MyDrive/Colab Notebooks/Yolo_Detection/dataset/images/val/ruin5_1_3.png: 640

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'ruin'}
 obb: None
 orig_img: array([[[155, 163, 180],
         [156, 160, 179],
         [139, 143, 161],
         ...,
         [189, 211, 230],
         [197, 215, 233],
         [201, 216, 235]],
 
        [[155, 163, 180],
         [153, 159, 178],
         [140, 144, 164],
         ...,
         [186, 208, 228],
         [193, 212, 232],
         [200, 215, 234]],
 
        [[146, 154, 172],
         [149, 155, 174],
         [136, 142, 161],
         ...,
         [180, 202, 222],
         [191, 210, 229],
         [200, 216, 234]],
 
        ...,
 
        [[ 78,  75,  79],
         [ 76,  73,  78],
         [ 72,  65,  76],
         ...,
         [220, 223, 246],
         [220, 222, 246],
         [208, 213, 236]],
 
        [[ 80,  76,  80],
         [ 79,  74,  79],
         [ 78,  71,  78],
         ...,
         [221, 225, 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
DATA_YAML = "/content/drive/MyDrive/Colab Notebooks/Yolo_Detection/dataset/data.yaml"